# Lab 3: Two Coupled Oscillators

> In this section we will consider a &ldquo;two mass three
> springs&rdquo; system.  The arrangement is similar to the system
> depicted in the Figure at the beginning of this worksheet, except the
> masses and springs are arranged vertically.  When the masses are in
> equilibrium, the gravitational force is compensated by forces due to
> the springs.  If we measure the displacements from the equilibrium
> position, we do not need to consider the effect of gravity.  We also
> added a disc to create a damping force, just like in the Worksheet #2
> experiment.
>
> In the first part of the lab we will look at the free oscillations;
> the second part is about forced oscillations.

The first code block installs the `uncertainties` package if it is not
installed in your environment.

In [ ]:
%pip install uncertainties

The second code block loads the packages that are needed by this
notebook.

In [ ]:
from __future__ import annotations  # noqa: F404

import gspread
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import scipy.optimize as opt
from IPython.display import Markdown
from google.auth import default
from google.colab import auth
from uncertainties import ufloat

The third code block, gets your Google account credentials and uses them
to access the spreadsheet containing your data.  (Make sure that the
account you are using to run this notebook has access to the Google
Sheets file containing your data.)

Replace `NAME OF GOOGLE SHEETS FILE` in the code block with the name of
your Google Sheets file.  (Remember to leave the quotation marks around
your file's name.)

The variables `p1ws` and `p2ws` refer to the &ldquo;Part1&rdquo; and
&ldquo;Part2&rdquo; worksheets (respectively) in your file.

In [ ]:
auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

gs = gc.open("NAME OF GOOGLE SHEETS FILE")

p1ws = gs.get_worksheet(0)
p2ws = gs.get_worksheet(1)

## Part 1: Free Oscillations

> 1. Start by initiating motion in the first normal (lower frequency)
>    mode by hand.  How can you do this?  Measure the frequency of
>    oscillation by using the motion sensor and fitting the measured
>    response.  Describe the motion of the two masses.

Here, we load your the first normal mode (angular) frequency from the
&ldquo;Part1&rdquo; worksheet.  The code expects your first normal mode
angular frequency to be in cell `B1` (row 1, column 2) and its
uncertainty to be in cell `D1` (row 1, column 4).  If this is not the
case, you must update the row and column numbers (the arguments to the
`cell()` method of `p1ws`) in the code below accordingly.

In [ ]:
part1 = {
    "omega_1": ufloat(
        p1ws.cell(1, 2).numeric_value,
        p1ws.cell(1, 4).numeric_value,
    ),
}
part1["f_1"] = part1["omega_1"] / (2 * np.pi)  # type: ignore
display(
    Markdown(
        fR"$\omega_1 = {part1['omega_1']:L} \, \text{{rad}} / \text{{s}} "
        + R"\implies "
        + fR"f_1 = {part1['f_1']:L} \, \text{{Hz}}$"
    )
)

> 2. Initiate motion in the second normal mode.  Measure the frequency
>    of oscillation by using the motion sensor and fitting the measured
>    response.

Here, we load your the second normal mode (angular) frequency from the
&ldquo;Part1&rdquo; worksheet.  The code expects your first normal mode
angular frequency to be in cell `B2` (row 2, column 2) and its
uncertainty to be in cell `D2` (row 2, column 4).  If this is not the
case, you must update the row and column numbers (the arguments to the
`cell()` method of `p1ws`) in the code below accordingly.

In [ ]:
part1["omega_2"] = ufloat(
    p1ws.cell(2, 2).numeric_value,
    p1ws.cell(2, 4).numeric_value,
)
part1["f_2"] = part1["omega_2"] / (2 * np.pi)  # type: ignore
display(
    Markdown(
        fR"$\omega_2 = {part1['omega_2']:L} \, \text{{rad}} / \text{{s}} "
        + R"\implies "
        + fR"f_2 = {part1['f_2']:L} \, \text{{Hz}}$"
    )
)

## Part 2: Forced Oscillations

> 3. Connect the driver to the signal generator.  Measure the amplitude
>    of the motion for the bottom mass as a function of the drive
>    frequency, scanning around the two normal mode frequencies you
>    determined before.  (Use $f = \omega / 2 \pi$ to calculate the
>    frequency from the angular frequency.)  Try to measure the
>    amplitude at about 20 different frequencies.  You may not want to
>    choose frequencies spaced by the same amount at all points on the
>    graph.  Here are some suggested frequencies to measure at:
>
>    $f_\text{drive} = f_1 + 0.20 \, \text{Hz}, f_1 + 0.15 \, \text{Hz},
>    f_1 + 0.10 \, \text{Hz}, f_1 + 0.08 \, \text{Hz},
>    f_1 + 0.06 \, \text{Hz}, f_1 + 0.05 \, \text{Hz},
>    f_1 + 0.04 \, \text{Hz}, f_1 + 0.03 \, \text{Hz},
>    f_1 + 0.02 \, \text{Hz}, f_1 + 0.01 \, \text{Hz},
>    f_1, f_1 - 0.01 \, \text{Hz}, f_1 - 0.02 \, \text{Hz},
>    f_1 - 0.03 \, \text{Hz}, f_1 - 0.04 \, \text{Hz},
>    f_1 - 0.05 \, \text{Hz}, f_1 - 0.06 \, \text{Hz},
>    f_1 - 0.08 \, \text{Hz}, f_1 - 0.10 \, \text{Hz},
>    f_1 - 0.15 \, \text{Hz}, f_1 - 0.20 \, \text{Hz}$ and similarly
>    around the second resonance frequency.

In the next two code blocks, we load your forced oscillation data for
the first normal mode and second normal mode.  The code expects the
`B3:I23` cell range of your &ldquo;Part2&rdquo; worksheet to contain
your data for the first normal mode and the `B25:I45` cell range (of the
same worksheet) to contain your data for the second normal mode.  If
that is not the case, you must update the range references in the blocks
below.  The code also assumes the variables are organized as follows:

1. column `B`: frequency, `f`
2. column `C`: amplitude, `A`
3. column `D`: amplitude uncertainty, `A_unc`
4. column `E`: angular frequency, `omega`
5. column `F`: angular frequency uncertainty, `omega_unc`
6. column `G`: `y`
7. column `H`: `y` uncertainty, `y_unc`
8. column `I`: &ldquo;f check,&rdquo; `f_check`

In [ ]:
df1 = pd.DataFrame(
    data=np.asarray(
        p2ws.batch_get(
            ["B3:I23"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "f",
        "A",
        "A_unc",
        "omega",
        "omega_unc",
        "y",
        "y_unc",
        "f_check",
    ],
)
display(df1)

In [ ]:
df2 = pd.DataFrame(
    data=np.asarray(
        p2ws.batch_get(
            ["B25:I45"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "f",
        "A",
        "A_unc",
        "omega",
        "omega_unc",
        "y",
        "y_unc",
        "f_check",
    ],
)
display(df2)

> 4. Graph your measured amplitude vs frequency.  It is recommended that
>    you make this graph before you leave the lab, because you want to
>    be sure that the peaks are approximately in the middle of the
>    frequency range of your measurements.

Now, we graph your amplitudes as a function of angular frequency for the
two normal modes.

It is your responsibility to make sure that the figures in your lab
report are neat, legible, etc., so you may wish to fine tune the figure
generation code below.  For instance, you may wish to adjust the figure
dimensions (see the calls to `fig.set_figureheight()` and
`fig.set_figurewidth()` below) and the value of the `capsize` parameter
(which controls the size of the error bar caps).

In [ ]:
fig, ax = plt.subplots()

fig.set_figheight(1.5 * fig.get_figheight())
fig.set_figwidth(2 * fig.get_figwidth())

fig.suptitle(R"First Normal Mode")
ax.set_title(R"Amplitude vs. Angular Frequency")
ax.set_xlabel(
    R"Angular Frequency, $\omega$ $\left[\text{rad} / \text{s}\right]$"
)
ax.set_ylabel(R"Amplitude, $A$ $\left[\text{m}\right]$")

ax.errorbar(
    x=df1.omega,
    y=df1.A,
    yerr=df1.A_unc,
    xerr=df1.omega_unc,
    linestyle="",
    capsize=2,
)

fig.tight_layout()

del fig, ax

In [ ]:
fig, ax = plt.subplots()

fig.set_figheight(1.5 * fig.get_figheight())
fig.set_figwidth(2 * fig.get_figwidth())

fig.suptitle(R"Second Normal Mode")
ax.set_title(R"Amplitude vs. Angular Frequency")
ax.set_xlabel(
    R"Angular Frequency, $\omega$ $\left[\text{rad} / \text{s}\right]$"
)
ax.set_ylabel(R"Amplitude, $A$ $\left[\text{m}\right]$")

ax.errorbar(
    x=df2.omega,
    y=df2.A,
    yerr=df2.A_unc,
    xerr=df2.omega_unc,
    linestyle="",
    capsize=2,
)

fig.tight_layout()

del fig, ax

> 5. Calculate the quantity $y = \omega^2 A^2$.  From the measured
>    amplitude $A$.  As we discussed in Worksheet #2, this quantity is
>    proportional to the power dissipation that follows a Lorentzian
>    frequency dependence of
>
>    $$ y = \cfrac{B}{4 \left(\omega_0 - \omega\right)^2 + \gamma^2} $$
>
>    (This expression is valid if the two resonance peaks are well
>    separated.)  Fit a Lorentzian to each resonance peak, and extract
>    the parameters $\omega_0$ and $\gamma$ for each peak.

First, we define our fit function, `y()`.

In [ ]:
def y(
    omega: npt.NDArray,
    omega_mode: float,
    gamma: float,
    b: float,
) -> npt.NDArray:
    return b / (4 * (omega_mode - omega) ** 2 + gamma**2)

Next, we fit your data, plot said data, plot your fit curve, and output
your fit parameters.  You may need to adjust the initial estimates of
the fit parameters (the `p0` arguments to `opt.curve_fit()`).

**Note:** The fits below do not account for uncertainty in the
independent variable.  You should consider whether or not it is
justifiable to neglect these uncertainties.  If you use this code (or
other code that does not consider uncertainties in the independent
variable) to fit your data, **you will need to justify this in your lab
report.**  If you determine that neglecting uncertainties in the
independent variable is not justifiable, you can use
[`scipy`'s orthogonal distance regression library](odr) to account for
uncertainties in both independent and dependent variables.

It is recommended that you use a metric for the &ldquo;goodness of
fit&rdquo; such as the [reduced chi-squared statistic](rcss) to gauge
the quality of your fits.

Contact your TAs if you need assistance.

[odr]: https://docs.scipy.org/doc/scipy/reference/odr.html
[rcss]: https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic

In [ ]:
part2 = {}

In [ ]:
fig, ax = plt.subplots()

fig.set_figheight(1.5 * fig.get_figheight())
fig.set_figwidth(2 * fig.get_figwidth())

fig.suptitle(R"First Normal Mode")
ax.set_title(R"$y$ vs. Angular Frequency")
ax.set_xlabel(
    R"Angular Frequency, $\omega$ $\left[\text{rad} / \text{s}\right]$"
)
ax.set_ylabel(
    R"$y = A^2 \omega^2$ "
    R"$\left[\text{m}^2 \, \text{rad}^2 / \text{s}^2\right]$"
)

popt1, pcov1 = opt.curve_fit(
    f=y,
    xdata=df1.omega,
    ydata=df1.y,
    sigma=df1.y_unc,
    p0=(
        part1["omega_1"].n,
        0.1,
        df1.y.max() * 0.02**2,
    ),
)
perr1 = np.sqrt(np.diag(pcov1))
part2["NM1"] = {
    "omega_1": ufloat(popt1[0], perr1[0]),
    "gamma": ufloat(popt1[1], perr1[1]),
    "B": ufloat(popt1[2], perr1[2]),
}

x_fit1 = np.linspace(
    df1.omega.min(),
    df1.omega.max(),
    100*(df1.omega.count()-1) + 1,
)

ax.errorbar(
    x=df1.omega,
    y=df1.y,
    yerr=df1.y_unc,
    xerr=df1.omega_unc,
    linestyle="",
    capsize=2,
    label=R"Data",
)
ax.plot(
    x_fit1,
    y(x_fit1,*popt1),
    color="k",
    linewidth=0.75,
    label=R"Fit",
    zorder=1.9,
)

ax.legend()

fig.tight_layout()

del fig, ax

display(
    Markdown(
        fR"$$\omega_1 = {part2['NM1']['omega_1']:L} \, "
        + R"\text{rad} / \text{s}$$" + "\n"
        + fR"$$\gamma = {part2['NM1']['gamma']:L} \, "
        + R"\text{rad} / \text{s}$$" + "\n"
        + fR"$$B = {part2['NM1']['B']:L} \, "
        + R"\text{m}^2 \, \text{rad}^4 / \text{s}^4$$"
    )
)

In [ ]:
fig, ax = plt.subplots()

fig.set_figheight(1.5 * fig.get_figheight())
fig.set_figwidth(2 * fig.get_figwidth())

fig.suptitle(R"Second Normal Mode")
ax.set_title(R"$y$ vs. Angular Frequency")
ax.set_xlabel(
    R"Angular Frequency, $\omega$ $\left[\text{rad} / \text{s}\right]$"
)
ax.set_ylabel(
    R"$y = A^2 \omega^2$ "
    R"$\left[\text{m}^2 \, \text{rad}^2 / \text{s}^2\right]$"
)

popt2, pcov2 = opt.curve_fit(
    f=y,
    xdata=df2.omega,
    ydata=df2.y,
    sigma=df2.y_unc,
    p0=(
        part1["omega_2"].n,
        0.1,
        df2.y.max() * 0.02**2,
    ),
)
perr2 = np.sqrt(np.diag(pcov2))
part2["NM2"] = {
    "omega_2": ufloat(popt2[0], perr2[0]),
    "gamma": ufloat(popt2[1], perr2[1]),
    "B": ufloat(popt2[2], perr2[2]),
}

x_fit2 = np.linspace(
    df2.omega.min(),
    df2.omega.max(),
    100*(df2.omega.count()-1) + 1,
)

ax.errorbar(
    x=df2.omega,
    y=df2.y,
    yerr=df2.y_unc,
    xerr=df2.omega_unc,
    linestyle="",
    capsize=2,
    label=R"Data",
)
ax.plot(
    x_fit2,
    y(x_fit2,*popt2),
    color="k",
    linewidth=0.75,
    label=R"Fit",
    zorder=1.9,
)

ax.legend()

fig.tight_layout()

del fig, ax

display(
    Markdown(
        fR"$$\omega_2 = {part2['NM2']['omega_2']:L} \, "
        + R"\text{rad} / \text{s}$$" + "\n"
        + fR"$$\gamma = {part2['NM2']['gamma']:L} \, "
        + R"\text{rad} / \text{s}$$" + "\n"
        + fR"$$B = {part2['NM2']['B']:L} \, "
        + R"\text{m}^2 \, \text{rad}^4 / \text{s}^4$$"
    )
)

## Lab Report

> In your formal lab report,
>
> - Summarize the experimental procedure,
> - Report the values of the normal mode frequencies (with uncertainty)
>   determined from the free oscillations.
> - Provide a graph of the measured amplitude vs. frequency.
> - Provide a graph of the evaluated $y = \omega^2 A^2$ vs. frequency,
>   together with the fitted Lorentzian curves.
> - Report the normal mode frequencies and damping parameters obtained
>   from the Lorentzian fits (with uncertainty).
> - Compare the resonance frequencies to the values determined in the
>   answer to Question 3 of this worksheet and discuss the reasons for
>   the imperfect agreement.